In [22]:
import os
from PIL import Image
from torch.utils.data import Dataset

class HandwritingDataset(Dataset):
    def __init__(self, root_dir, transform=None):
        self.root_dir = root_dir
        self.transform = transform
        self.image_paths = []
        self.labels = []
        self.label_mapping = {}
        current_label = 0

        # Valid image extensions
        valid_extensions = ('.png', '.jpg', '.jpeg', '.bmp', '.tiff', '.gif')

        for label in os.listdir(root_dir):
            label_path = os.path.join(root_dir, label)
            if os.path.isdir(label_path):
                self.label_mapping[label] = current_label
                current_label += 1
                for img_name in os.listdir(label_path):
                    img_path = os.path.join(label_path, img_name)
                    if os.path.isfile(img_path) and img_path.endswith(valid_extensions):
                        self.image_paths.append(img_path)
                        self.labels.append(self.label_mapping[label])

        # Debugging output
        print(f"Loaded {len(self.image_paths)} images with labels {set(self.labels)}")

    def __len__(self):
        return len(self.image_paths)

    def __getitem__(self, idx):
        img_path = self.image_paths[idx]
        image = Image.open(img_path).convert("L")  # Convert image to grayscale
        label = self.labels[idx]

        if self.transform:
            image = self.transform(image)

        return image, label

In [23]:
from torchvision import transforms

transform = transforms.Compose([
    transforms.Resize((128, 128)),  # Resize images to 128x128
    transforms.Grayscale(num_output_channels=1),  # Convert to grayscale
    transforms.ToTensor(),  # Convert image to PyTorch tensor
    transforms.Normalize((0.5,), (0.5,))  # Normalize the image tensor
])

In [24]:
import torch
import torch.nn as nn

# Assuming `model` is defined elsewhere
sample_input = torch.randn(1, 1, 256, 256)  # Single grayscale image, 256x256
conv_output = model.conv1(sample_input)
conv_output = model.conv2(conv_output)
conv_output = nn.functional.max_pool2d(conv_output, 2)
print(conv_output.shape)  # Check output shape for debugging

torch.Size([1, 32, 126, 126])


In [27]:
import os

# Define your HandwritingDataset and DataLoader
root_dir = "./A"  # Adjust this if the "A" folder is in a different location

# Define a function to load only image files
def is_image_file(filename):
    return filename.lower().endswith((".png", ".jpg", ".jpeg"))

# Adjust HandwritingDataset to ignore non-image files
class HandwritingDataset(Dataset):
    def __init__(self, root_dir, transform=None):
        self.root_dir = root_dir
        self.transform = transform
        # Filter to include only image files
        self.image_files = [f for f in os.listdir(root_dir) if is_image_file(f)]
        print(f"Loaded {len(self.image_files)} images with labels set")  # Debugging output

    def __len__(self):
        return len(self.image_files)

    def __getitem__(self, idx):
        img_name = os.path.join(self.root_dir, self.image_files[idx])
        image = Image.open(img_name).convert("L")  # Convert to grayscale if needed
        if self.transform:
            image = self.transform(image)
        label = 1  # Assign label based on your specific setup
        return image, label

# Map labels {1, 2} -> {1, 0}
def binary_label_mapping(labels):
    return torch.where(labels == 2, torch.tensor(0), labels)

# Create DataLoader
dataset = HandwritingDataset(root_dir=root_dir, transform=transform)
train_loader = DataLoader(dataset, batch_size=32, shuffle=True)

# Debugging: Check if DataLoader works correctly
# for images, labels in train_loader:
#     # Apply label mapping
#     labels = binary_label_mapping(labels)
#     print(images.shape, labels)
#     break  # Remove this break after debugging


Loaded 6 images with labels set


NameError: name 'DataLoader' is not defined

In [28]:
import torch
import torch.nn as nn
import torch.optim as optim

class SimpleCNN(nn.Module):
    def __init__(self):
        super(SimpleCNN, self).__init__()
        self.conv1 = nn.Conv2d(1, 16, 3, 1)
        self.conv2 = nn.Conv2d(16, 32, 3, 1)
        self.fc1 = nn.Linear(32 * 30 * 30, 64)  # Adjust based on the actual output shape
        self.fc2 = nn.Linear(64, 1)

    def forward(self, x):
        x = nn.functional.relu(self.conv1(x))
        x = nn.functional.max_pool2d(x, 2)
        x = nn.functional.relu(self.conv2(x))
        x = nn.functional.max_pool2d(x, 2)
        
        # Check the shape of x dynamically
        x = x.view(x.size(0), -1)  # Flatten the tensor
        x = nn.functional.relu(self.fc1(x))
        x = self.fc2(x)
        return torch.sigmoid(x)

# Initialize the model
model = SimpleCNN()


In [29]:
# Initialize the model, criterion, and optimizer
model = SimpleCNN()
criterion = nn.BCELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

In [30]:
num_epochs = 10

for epoch in range(num_epochs):
    for images, labels in train_loader:
        # Ensure images are transformed and labels are converted
        labels = torch.tensor([1 if label == 'A' else 0 for label in labels]).float()

        # Forward pass
        outputs = model(images)
        outputs = outputs.squeeze()
        loss = criterion(outputs, labels)

        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}")


NameError: name 'train_loader' is not defined

In [31]:
# with torch.no_grad():
#     for images, labels in train_loader:
#         images = images
#         outputs = model(images)
#         predictions = (outputs.squeeze() > 0.5).int()  # Threshold at 0.5
#         print(f"Predicted: {predictions}, Actual: {labels}")

criterion = nn.CrossEntropyLoss()  # For multi-class classification


with torch.no_grad():
    for images, labels in train_loader:
        outputs = model(images)
        predictions = (outputs.squeeze() > 0.5).int()  # Apply threshold 0.5 for binary classification
        print(f"Predicted: {predictions}, Actual: {labels}")

NameError: name 'train_loader' is not defined

In [17]:
torch.save(model.state_dict(), "Character_A.pth")


In [34]:
from PIL import Image

# Load an image and apply transformations
img_path = 'C:/Users/sansk/Desktop/our_project_volume_II/CNN_(Convulation_Neural_Network)/A/2.jpg'  # Replace with your test image path
img = Image.open(img_path).convert('L')
img = transform(img).unsqueeze(0)  # Add batch dimension

# Set model to evaluation mode
model.eval()
with torch.no_grad():
    output = model(img)  # Model output for a single image
    prediction = output.item()  # Get scalar prediction

    # Apply threshold
    if prediction >= 0.5:
        print("Predicted: A")
    else:
        print("Predicted: Not A")

Predicted: A


In [19]:
import torch
from PIL import Image
from torchvision import transforms

# Path to the image
img_path = 'C:/Users/sansk/Desktop/our_project_volume_II/CNN_(Convulation_Neural_Network)/test_data/img5.jpg'
img = Image.open(img_path).convert('L')
img = transform(img).unsqueeze(0)  # Add batch dimension

# Set model to evaluation mode
model.eval()
with torch.no_grad():
    output = model(img)  # Output shape: [1, 1] for a single image
    prediction = output.item()  # Get scalar prediction

    # Decide based on a threshold (e.g., 0.5 for binary classification)
    if prediction >= 0.7:
        print("Predicted: A")
    else:
        print("Predicted: Not A")

Predicted: Not A
